#compare gradient calculation with convolution (OpenCV) and ft (SciPy)

##image filtering (convolution): calculate first derivative in x- and y-direction

`cv2.Sobel(src, ddepth, xorder, yorder, ksize) → dst`

"` xorder=1, yorder=0, ksize=3 `" leads to the kernel: $\begin{bmatrix}
-1 & -2 & -1\\
0 & 0 & 0\\
1 & 2 & 1
\end{bmatrix}$

and "`xorder=0, yorder=1, ksize=3`" leads to: $\begin{bmatrix}
-1 & 0 & 1\\
-2 & 0 & 2\\
-1 & 0 & 1
\end{bmatrix}$

witch combines gaussian smoothing and differentiation

the resulting images are 64bit float images (`ddepth=cv2.CV_64F`)

see: http://docs.opencv.org/modules/imgproc/doc/filtering.html#sobel

In [1]:
# make inline plots and images (built-in magic command of IPython)
#%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('trian90.bmp',0)
print('shape of gray image (height, width): ' + str(img.shape))

sobelx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3)
sobely = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)

plt.figure
plt.subplot(2,2,1),plt.imshow(img,cmap = 'gray')
plt.title('Original'), plt.xticks([]), plt.yticks([])
plt.subplot(2,2,3),plt.imshow(sobelx,cmap = 'gray')
plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
plt.subplot(2,2,4),plt.imshow(sobely,cmap = 'gray')
plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])

shape of gray image (height, width): (300, 400)


(<matplotlib.text.Text at 0xab86f78c>,
 ([], <a list of 0 Text xticklabel objects>),
 ([], <a list of 0 Text yticklabel objects>))

**calculate absolut value (magnitude) and direction (phase) of the gradient**

http://docs.opencv.org/modules/core/doc/operations_on_arrays.html?highlight=ft#magnitude

http://docs.opencv.org/modules/core/doc/operations_on_arrays.html?highlight=ft#phase

for histogram calculation see: http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_imgproc/py_histograms/py_histogram_begins/py_histogram_begins.html

`cv2.calcHist(images, channels, mask, histSize, ranges) → hist`

In [2]:
print(np.min(sobelx))
print(np.max(sobelx))
print(np.min(sobely))
print(np.max(sobely))

-428.0
276.0
-295.0
398.0


In [3]:
# calculate magnitude and phase
mag = cv2.magnitude(sobelx, sobely)
phase = cv2.phase(sobelx, sobely, angleInDegrees = 1)

In [4]:
# analyse magnitude values to find threshold
histmag = cv2.calcHist([mag],[0],None,[256],[0, int(mag.max())])
MAG_THRESH = 100

# use magnitude as mask for the phase (scaling!!)
mask = np.uint8(255.0 / mag.max() * mag)
ret, mask = cv2.threshold(mask, MAG_THRESH, 1, cv2.THRESH_BINARY)
mphase = cv2.multiply(mask, phase, dtype = cv2.CV_32F)

# look for the angles in gradient on strong edges
hist = cv2.calcHist([phase],[0],None,[360],[0,359])

# find masked values and set them to -1, calculate histogram from 0...359
mphase[mask == 0] = -1
histmphase = cv2.calcHist([mphase],[0],None,[360],[0,359])

# save histogram as text
np.savetxt('magnitude_hist.txt', histmag, fmt = '%04.1f', header = 'histogram values of magnitude')
np.savetxt('phase_hist.txt', hist, fmt = '%d', header = 'histogram values of phase, bins in degree')
np.savetxt('mphase.txt', mphase, fmt = '%d', header = 'masked values of phase in degree')
np.savetxt('mphase_hist.txt', histmphase, fmt = '%d')

In [5]:
print('shape masked phase: '+str(mphase.shape))
print(type(mphase))

shape masked phase: (300, 400)
<type 'numpy.ndarray'>


In [6]:
# show images and results
plt.figure()
plt.subplot(2,3,1)
plt.imshow(mag, cmap = 'gray'), plt.xticks([]), plt.yticks([])
plt.title('magnitude')

plt.subplot(2,3,2)
plt.imshow(mask, cmap = 'gray')
plt.title('mask'), plt.xticks([]), plt.yticks([])

plt.subplot(2,3,4)
plt.plot(histmag)
plt.ylim(0,200)
plt.plot([MAG_THRESH, MAG_THRESH],[0, 1000],'-r')
plt.title('histogram of magnitude (with threshold for mask)')

plt.subplot(2,3,3)
plt.imshow(phase, cmap = 'gist_rainbow', vmin = 0, vmax = 360)
plt.title('phase'), plt.xticks([]), plt.yticks([])
plt.colorbar();

# plot masked phase and histogram of phase
plt.figure()
plt.subplot(1,2,1)
mphase[mask == 0] = np.nan
plt.imshow(mphase, cmap = 'gist_rainbow', vmin = 0, vmax = 360)
plt.title('mask * phase'), plt.xticks([]), plt.yticks([])
plt.colorbar();

plt.subplot(1,2,2)
plt.plot(hist / max(hist), color = 'b', linewidth = 1)
plt.plot(histmphase / max(histmphase), color = 'g', linewidth = 2)
plt.title('histogram of phase (normalized)')

#plt.show()

##denoise with gaussian psf and calculate gradient with fourier transformation


$$\frac{df}{dx}=ik_x \cdot FT(f) \quad k_x = 2 \pi u$$

In [7]:
import math

# get frequency scale
[cy,cx]=img.shape
umax = cx / 2.0
vmax = cy / 2.0

u = np.fft.ifftshift(np.linspace(-umax, +umax, cx, endpoint = False))
v = np.fft.ifftshift(np.linspace(-vmax, +vmax, cy, endpoint = False))

# gaussian for noise reduction
sigmaX = 80.0
sigmaY = 60.0
U, V = np.meshgrid(u, v)
gauss = 1.0/(2.0 * math.pi * sigmaX * sigmaY) * np.exp(-(np.square(U / sigmaX) / 2 + np.square(V / sigmaY) / 2))

# the gradient funktion
gradX = np.matrix(2.0 * u * math.pi * 1j)
gradY = np.transpose(np.matrix(2.0 * v * math.pi * 1j))

# fft to convert the image to freq domain and get FT(d/dx) and FT(d/dy)
FIMG = np.fft.fft2(img)
FIMG = np.multiply(FIMG, gauss)
dxFIMG = np.multiply(FIMG, gradX)
dyFIMG = np.multiply(FIMG, gradY)

# inverse fft to get the image back 
dxImg = np.fft.ifft2(dxFIMG)
dyImg = np.fft.ifft2(dyFIMG)

In [8]:
print(np.shape(u))
print(np.shape(v))

print(np.shape(gradX))
print(np.shape(gradY))

print(np.min(dxImg.real))
print(np.max(dxImg.real))
print(np.min(dyImg.real))
print(np.max(dyImg.real))

print(np.min(dxImg.imag))
print(np.max(dxImg.imag))
print(np.min(dyImg.imag))
print(np.max(dyImg.imag))

print(np.shape(gauss[0:,0]))
print(np.shape(gauss[0,0:]))

(400,)
(300,)
(1, 400)
(300, 1)
-0.64629630399
0.429197578111
-0.34462099027
0.457760769818
-0.000512429355584
0.000512429355584
-0.00027158836358
0.00027158836358
(300,)
(400,)


In [9]:
# show images and results
plt.figure
plt.subplot(2,3,1),plt.imshow(img,cmap = 'gray')
plt.title('Original'), plt.xticks([]), plt.yticks([])

plt.subplot(2,3,2)
mmin=np.min([dxImg.real, dyImg.real])
mmax=np.max([dxImg.real, dyImg.real])
plt.imshow(dxImg.real, cmap = 'gray', vmin = mmin, vmax = mmax), plt.xticks([]), plt.yticks([])
plt.title('real part of df/dx')
plt.subplot(2,3,3)
plt.imshow(dyImg.real, cmap = 'gray', vmin = mmin, vmax = mmax), plt.xticks([]), plt.yticks([])
plt.title('real part of df/dy')

plt.subplot(2,3,4)
plt.imshow(cv2.magnitude(dxImg.real, dyImg.real), cmap = 'gray'), plt.xticks([]), plt.yticks([])
plt.title('magnitude of gradient')
plt.subplot(2,3,5)
plt.imshow(cv2.phase(dxImg.real, dyImg.real, angleInDegrees = 1), cmap = 'gist_rainbow', vmin = 0, vmax = 360), plt.xticks([]), plt.yticks([])
plt.title('phase of gradient')
plt.colorbar();

# plot slices in u and v from gaussian
plt.figure()
plt.plot(np.fft.fftshift(v), np.fft.fftshift(gauss[0:, 0])/max(gauss[0:, 0]))
plt.plot(np.fft.fftshift(u), np.fft.fftshift(gauss[0, 0:])/max(gauss[0, 0:]))
plt.plot(np.fft.fftshift(u), np.fft.fftshift(np.fft.fft([1, 2, 1], len(u))) / 4.0)
plt.title('gaussian filter')

plt.show()

/usr/lib/python2.7/dist-packages/numpy/core/numeric.py:460: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
